In [423]:
from contextlib import contextmanager
from threading  import Lock, Thread
import random
from time import sleep
import operator as _

class SharedExclusive(object):

    def __init__(self):
        self.write_lock_obj = Lock()
        self.num_r = 0 # Number of read_locks

    def r_acquire(self):
        print("\nRead_lock Called:")
        if self.write_lock_obj.locked():
            print("Wait for write_lock")
            
            # waiting until write_lock released...
            while self.write_lock_obj.locked():
                print('.',end="")
            print("Read lock")
            self.num_r += 1
        else:
            print("Read lock")
            self.num_r += 1

    def r_release(self):
        if self.num_r > 0:
            print("Release read lock")
            self.num_r -= 1

    @contextmanager
    def r_locked(self):
        try:
            self.r_acquire()
            yield
        finally:
            self.r_release()

    def w_acquire(self):
        print("\nWrite_lock Called:")
        if self.num_r:
            print("Wait for read_lock")
            
            # waiting until read_lock released...
            while self.num_r:
                print('*',end="")
            print('Write lock')
            self.write_lock_obj.acquire()
            
        elif self.write_lock_obj.locked():
            print("Wait for write_lock")
            self.write_lock_obj.acquire()
        else:
            print('Write lock')
            self.write_lock_obj.acquire()

    def w_release(self):
        if self.write_lock_obj.locked():
            print('Release write lock')
            self.write_lock_obj.release()

    @contextmanager
    def w_locked(self):
        try:
            self.w_acquire()
            yield
        finally:
            self.w_release()
            
    def create_local_variable(self, block):
        variable = 0
        ldic = locals()
        try:
            exec(f"variable = self.{block['variable']}", ldic)
            # print(f"variable {block['variable']} already exists!")
        except AttributeError:
            exec(f"self.{block['variable']} = 0")
            exec(f"variable = self.{block['variable']}", ldic)
            # print(f"variable {block['variable']} created!")
        finally:
            variable = ldic['variable']
        return variable
    
            
class Concurrency:
    def __init__(self, locks):
        self.locks = locks
    
    def create_transaction(self, transaction):
        for block in transaction:
            operation_name = block['operations'][0]
            lock_class = self.locks.get(block['variable']) # Fetch variable object from sharedExclusive class.
            if operation_name == 'read_item':
                with lock_class.r_locked():
                    variable = lock_class.create_local_variable(block)
                    print(variable)
                    random_number = random.uniform(0.1,1.5)
                    print(f"~~ Sleep for {random_number:.2f} s")
                    sleep(random_number)
                    
            elif operation_name == 'write_item':
                operation_task = block['operations'][1]
                with lock_class.w_locked():
                    variable = lock_class.create_local_variable(block)
                    print(variable)
                    operator, operand = operation_task.split()
                    mapper = {
                        '+' : _.add,
                        '-' : _.sub,
                        '*' : _.mul,
                        '/' : _.truediv,
                    }
                    func = mapper.get(operator, None)
                    if func:
                        variable = func(variable, float(operand))
                    random_number = random.uniform(0.1,1.5)
                    print(f"~~ Sleep for {random_number:.2f} s")
                    sleep(random_number)
                    exec(f"lock_class.{block['variable']} = variable")

In [434]:
def create_and_run_threads(schedule, concurrency):
    for index, transaction in enumerate(schedule):
        exec(f"t{index} = Thread(target=concurrency.create_transaction, args=({transaction},))")
    index += 1
    for i in range(index):
        exec(f"t{i}.start()")
    for i in range(index):
        exec(f"t{i}.join()")

def get_variables(schedule):
    vars = []
    for transaction in schedule:
        for block in transaction:
            vars.append(block['variable'])
    # Remove duplicates
    return list(set(vars))

def create_lock_objects(vars, SharedExclusive):
    objects = {}
    for var in vars:
        ldic = locals()
        exec(f"objects[var] = SharedExclusive()", ldic)
    return ldic['objects']

def print_variables(locks):
    print('\nAll used variables and values:')
    for var, obj in locks.items():
        print(var, "=", getattr(obj, var))
        
def main():
    transaction = [
        {
            'variable': "y",
            'operations': [
                'write_item', 
                '+ 10'
            ]
        },
        {
            'variable': "x",
            'operations': [
                'write_item', 
                '+ 10'
            ]
        },
    ]
    # for simplicity we assume all transactions are same.
    schedule = [transaction for _ in range(5)]
    vars = get_variables(schedule)
    locks = create_lock_objects(vars, SharedExclusive)
    concurrency = Concurrency(locks)
    create_and_run_threads(schedule, concurrency)
    print_variables(locks)

In [435]:
main()


Write_lock Called:
Write lock
0
~~ Sleep for 1.06 s

Write_lock Called:
Write_lock Called:
Wait for write_lock

Wait for write_lock

Write_lock Called:
Wait for write_lock

Write_lock Called:
Wait for write_lock
Release write lock

Write_lock Called:
Write lock
0
~~ Sleep for 0.38 s
10.0
~~ Sleep for 0.66 s
Release write lock
Release write lock

Write_lock Called:
Write lock
10.0
~~ Sleep for 1.24 s
20.0
~~ Sleep for 1.27 s
Release write lock
Release write lock

Write_lock Called:
30.0Write lock
20.0

~~ Sleep for 0.20 s
~~ Sleep for 0.30 s
Release write lock
Release write lock

Write_lock Called:
Write lock
40.030.0
~~ Sleep for 0.94 s
~~ Sleep for 1.43 s

Release write lock

Write_lock Called:
Wait for write_lock
Release write lock
40.0
~~ Sleep for 1.02 s
Release write lock

All used variables and values:
y = 50.0
x = 50.0
